In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:10<00:00, 91.2MB/s]
100% 1.06G/1.06G [00:10<00:00, 106MB/s] 


In [3]:
import zipfile 
zippedfile = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zippedfile.extractall('/content')
zippedfile.close()

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D , Flatten , Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

In [5]:
model = Sequential()
""" now we add a convolution to our first layer to extract features from the pics that we add the so we need to use Conv2D funcion with a specific number
of filters (karnels) this filters are slide into the pic and extarct features (increse number of filter = increase the ability of extract complex features
from pics and also may conduct to an overfitting , after this each filter generat a matrix (3x3) in our case to be used in weights channels and help neural 
network be more precisable like the bias value """
model.add(Conv2D(32,(3,3),input_shape=(256,256,3),activation="relu")) #inpute_shpe --> 64x64 dimension --> 3 vlues represent the color used (principal colors red , green , blue ) in grey scale there is only 1 value
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) #use it to reduce the number of feature map in this layer 
model.add(Flatten())# reshape the tensor into a one-dimentionel (after it was 3d) so now we need a hidden layer to make the output of this into it 
model.add(Dense(units=128 , activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(units=1 , activation='sigmoid'))#here w use a segmoid function bcs the prediction will be 1 or 0 (cat or dog )

In [6]:
""" optimizer how reverse propagation adam is the most used and work good with large data ,  for the loss and metrics visit keras documentation"""
model.compile(optimizer="adam",loss='binary_crossentropy',metrics =['accuracy']) 

In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale=1./255 , shear_range=0.2,zoom_range=0.2,horizontal_flip=True) #bcs we have 255 color 
train_set = train_gen.flow_from_directory('/content/train',shuffle=True,target_size = (256,256),batch_size=32 , class_mode ="binary")

Found 20000 images belonging to 2 classes.


In [11]:
test_gen = ImageDataGenerator(rescale=1./255 , shear_range=0.2,zoom_range=0.2,horizontal_flip=True) #bcs we have 255 color 
test_set = train_gen.flow_from_directory('/content/test',shuffle=True,target_size=(256,256),batch_size=32 , class_mode ="binary")

Found 5000 images belonging to 2 classes.


In [12]:
model.fit_generator(train_set,steps_per_epoch=500,epochs=5,validation_data=test_set,validation_steps=155)
""" about the number that we choose in steps per epochs , epochs , validaition_steps ... etc so 
 in the first we choose a batch size in our case we have small data so after process 32 data examples the model update 
 his weights that why batch_size=32 , for the steps per epoch mark that in evry epoch we will process the entire dataset
 so 332*32=10624= len(dataset) that means that we are updating our weights 332 time in each epoch """

<ipython-input-12-767a5eb3ac33>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_set,steps_per_epoch=500,epochs=5,validation_data=test_set,validation_steps=155)


Epoch 1/5
500/500 [==============================] - 4334s 9s/step - loss: 0.8641 - accuracy: 0.5774 - val_loss: 0.6461 - val_accuracy: 0.6169
Epoch 2/5
500/500 [==============================] - 4296s 9s/step - loss: 0.6578 - accuracy: 0.6056 - val_loss: 0.6688 - val_accuracy: 0.6121
Epoch 3/5
500/500 [==============================] - 4212s 8s/step - loss: 0.6329 - accuracy: 0.6478 - val_loss: 0.6039 - val_accuracy: 0.6728
Epoch 4/5
500/500 [==============================] - 4203s 8s/step - loss: 0.5983 - accuracy: 0.6834 - val_loss: 0.5656 - val_accuracy: 0.7133
Epoch 5/5
500/500 [==============================] - 4216s 8s/step - loss: 0.5716 - accuracy: 0.7061 - val_loss: 0.5399 - val_accuracy: 0.7278


' about the number that we choose in steps per epochs , epochs , validaition_steps ... etc so \n in the first we choose a batch size in our case we have small data so after process 32 data examples the model update \n his weights that why batch_size=32 , for the steps per epoch mark that in evry epoch we will process the entire dataset\n so 332*32=10624= len(dataset) that means that we are updating our weights 332 time in each epoch '

In [23]:
import numpy as np
from keras.preprocessing import image
import keras.utils as image
test_image = image.load_img('/content/121594.jpg',target_size=(256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)
print(train_set.class_indices)
if result[0][0]==1 : #the model can made wrong prediction with 30%-50% just e need t traine more  
  print("dog")
else :
  print("cat")

1/1 [==============================] - 0s 108ms/step
{'cats': 0, 'dogs': 1}
cat
